In [ ]:
!pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import requests
import os

#All Languages audio

In [ ]:
%%capture

!pip install boto3
!pip install beautifulsoup4
import boto3
import getpass
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import os
import json
import csv
import pickle

In [ ]:
access_key = getpass.getpass('key')
secret = getpass.getpass('secret')

In [ ]:
session = boto3.session.Session()
client = session.client('s3',
                        region_name='us-east-1',
                        aws_access_key_id=access_key,
                        aws_secret_access_key=secret)

In [ ]:
s3 = boto3.resource('s3',                        
                    region_name='us-east-1',
                    aws_access_key_id=access_key,
                    aws_secret_access_key=secret)
bucket = s3.Bucket('bloom-speech')
#len(list(bucket.objects.all()))

In [ ]:
for obj in bucket.objects.all():
  fname = obj.key
  s3.Object('bloom-speech',fname).Acl().put(ACL='public-read')

In [ ]:
for obj in tqdm(bucket.objects.limit(25000)):
  fname = obj.key
  if fname.startswith('bloom_downloads') and (fname.endswith('.htm') or fname.endswith('meta.json')):
    os.makedirs(os.path.dirname(fname), exist_ok=True)
    client.download_file('bloom-raw-data', fname, fname)

In [ ]:
import re
import json
import csv

books = os.listdir('/content/bloom_downloads')
licenses = set()

for book in tqdm(books):
  metapath = os.path.join('/content','bloom_downloads', book, 'meta.json')
  if os.path.isfile(metapath):
    with open(metapath, 'rb') as jsonfile:
      meta = json.load(jsonfile)
      licenses.add(meta.get('license'))
      langs = meta.get('language-display-names',[])
  bookpath = os.path.join('/content','bloom_downloads', book)
  bookfiles = os.listdir(bookpath)
  for lang in langs:
    for f in bookfiles:
      if f.endswith('.htm'):
        with open(os.path.join(bookpath,f),'rb') as htmlfile:
          soup = BeautifulSoup(htmlfile, 'html.parser')
        for hit in soup.findAll(attrs={'lang': lang}):
          print(lang)
          for sent in hit.findAll(attrs={'class' : 'audio-sentence'}):
            #print(sent)
            print(sent.get_text().strip())
          break

In [ ]:
for obj in tqdm(bucket.objects.all()):
  fname = obj.key
  if fname.startswith('bloom_downloads') and (fname.endswith('.htm') or fname.endswith('meta.json')):
    os.makedirs(os.path.dirname(os.path.join('/content/drive/MyDrive/bloom',fname)), exist_ok=True)
    client.download_file('bloom-raw-data', fname, os.path.join('/content/drive/MyDrive/bloom',fname))

In [ ]:
def clean_title(title):
    invalid = '<>:"/\|?*'
    removes = '()'
    for char in invalid:
      title = title.replace(char, " ")
    for char in removes:
      title = title.replace(char, '')
    if len(title) > 50:
      title = title[0:50]
    return(title)

In [ ]:
import pickle

with open('/content/drive/MyDrive/bloom/valid_urls.pkl', 'rb') as pklfile:
          urls = pickle.load(pklfile)

In [ ]:
import pickle

with open('/content/drive/MyDrive/bloom/valid_urls.pkl', 'rb') as pklfile:
          urls = pickle.load(pklfile)

In [ ]:
urls = []

In [ ]:
import re
import json
import csv
import pickle

bloom_dir = '/content/drive/MyDrive/bloom/bloom_downloads'
books = os.listdir(bloom_dir)
bloom_audio = {}

for book in tqdm(books):
  metapath = os.path.join(bloom_dir, book, 'meta.json')
  if os.path.isfile(metapath):
    with open(metapath, 'rb') as jsonfile:
      meta = json.load(jsonfile)
      if meta.get('language-display-names') and meta.get('license') not in ['custom', 'ask', None]: 
        for lang in meta['language-display-names'].keys():
          if f'talkingBook:{lang}' in meta['features']:
            #cak_tbs.append(book)
            downsrc = meta.get('downloadSource')
            if downsrc == None:
              print(f'{book} missing downloadSource')
            try:
              titles_dict = json.loads(meta['allTitles'].replace("\n", " "))
            except:
              print(meta['allTitles'], type(meta['allTitles']))
              break
            if titles_dict.get(lang):
              title = titles_dict.get(lang)
              title = clean_title(title)
              title_path = os.path.join(bloom_dir, book, title.strip() + '.htm')
              if not os.path.isfile(title_path):
                #print(f'{title_path} not found in {os.listdir(os.path.join("/content","bloom_downloads", book))} trying book')
                title = book
                title = clean_title(title)
                title_path = os.path.join(bloom_dir, book, title.strip() + '.htm')
              if not os.path.isfile(title_path):
                #print(f'{title_path} also not found')
                titles = [x for x in os.listdir(os.path.join(bloom_dir, book)) if x.endswith('.htm')]
                if len(titles)>0:
                  title=titles[0]
                  title_path = os.path.join(bloom_dir, book, title)
            else:
              print(f'no {lang} title for {book}')
              title = book
              title = clean_title(title)
              title_path = os.path.join(bloom_dir, book, title.strip() + '.htm')

            try:
              with open(title_path,'rb') as htmlfile:
                soup = BeautifulSoup(htmlfile, 'html.parser')
              #with open('/content/drive/MyDrive/bloom/all-lang-files.csv', 'a', encoding='utf-16') as csvfile:
                #writer = csv.writer(csvfile, dialect='excel', delimiter='\t')
              for hit in soup.findAll(attrs={'lang': lang, 'class' : 'audio-sentence'}):
                fileid = hit.get_attribute_list('id')[0]
                text = hit.get_text().strip()                 
                if text != '':
                  url = f'https://s3.amazonaws.com/bloomharvest/{downsrc}/bloomdigital/audio/{fileid}.mp3'
                  if url not in urls:
                    page = requests.get(url)
                    if page.status_code == 200:
                      if not bloom_audio.get(lang):
                        bloom_audio[lang]=[] 
                      bloom_audio[lang].append({"book": book, "url": url, "text": text})
                      urls.append(url)
                  else:
                    if {"book": book, "url": url, "text": text} not in bloom_audio.get(lang,[]):
                      if not bloom_audio.get(lang):
                        bloom_audio[lang]=[] 
                      bloom_audio[lang].append({"book": book, "url": url, "text": text})
              for lhit in soup.findAll(attrs={'lang':lang}):
                for hit in lhit.findAll(attrs={'class' : 'audio-sentence'}):
                  fileid = hit.get_attribute_list('id')[0]
                  text = hit.get_text().strip()                 
                  if text != '':
                    url = f'https://s3.amazonaws.com/bloomharvest/{downsrc}/bloomdigital/audio/{fileid}.mp3'
                    if url not in urls:
                      page = requests.get(url)
                      if page.status_code == 200:
                        if not bloom_audio.get(lang):
                          bloom_audio[lang]=[] 
                        bloom_audio[lang].append({"book": book, "url": url, "text": text})
                        urls.append(url)
                    else:
                      if {"book": book, "url": url, "text": text} not in bloom_audio.get(lang,[]):
                        if not bloom_audio.get(lang):
                          bloom_audio[lang]=[] 
                        bloom_audio[lang].append({"book": book, "url": url, "text": text})

            except:
              print(print(f'{title_path} failed download'))
              raise

        with open('/content/drive/MyDrive/bloom/valid_urls.pkl', 'wb') as pklfile:
          pickle.dump(urls, pklfile)
                      
          #print(book)

      

        

In [ ]:
def parents_havelang(tag):
  answer = False
  for parent in tag.parents:
    if parent.attrs.get('lang') and parent.attrs.get('lang') != '*':
      answer = True
      break
  return(answer) 

In [ ]:
import re
import json
import csv
import pickle

bloom_dir = '/content/drive/MyDrive/bloom/bloom_downloads'
books = os.listdir(bloom_dir)
bloom_audio = {}
badurls = []
prob_books = {}
statcodes = {}

tb_set = set()
tb_lds = {}
missing_lang = 0
has_lang = 0
ml_books = set()
hl_books = set()
lang_set = set()

for book in tqdm(books):
  metapath = os.path.join(bloom_dir, book, 'meta.json')
  if os.path.isfile(metapath):
    with open(metapath, 'rb') as jsonfile:
      meta = json.load(jsonfile)
      if meta.get('features') and meta.get('license') not in ['custom', 'ask', None]:
        for feat in meta['features'] :
          if feat.startswith('talkingBook'):
            downsrc = meta.get('downloadSource')
            if downsrc == None:
              print(f'{book} missing downloadSource')

            titles = [x for x in os.listdir(os.path.join(bloom_dir, book)) if x.endswith('.htm')]
            if titles == []:
              print(f'{book} lacks .htm')
              print(os.listdir(os.path.join(bloom_dir, book)))
            else:
              title=titles[0]
              title_path = os.path.join(bloom_dir, book, title)

            try:
              with open(title_path,'rb') as htmlfile:
                soup = BeautifulSoup(htmlfile, 'html.parser')

              for hit in soup.find_all(attrs={'class' : 'audio-sentence'}):
                if hit != []:
                  if hit.attrs.get('lang') and hit.attrs.get('lang')!= '*':
                    lang = hit.attrs.get('lang')
                    fileid = hit.get_attribute_list('id')[0]
                    text = hit.get_text().strip()
                    has_lang += 1
                    lang_set.add(lang)
                    hl_books.add(book)
                  elif parents_havelang(hit): 
                    for parent in hit.parents:
                      if parent.attrs.get('lang') and parent.attrs.get('lang') != '*':
                        lang = parent.attrs.get('lang')
                        break
                    fileid = hit.get_attribute_list('id')[0]
                    text = hit.get_text().strip()
                    has_lang += 1
                    lang_set.add(lang)
                    hl_books.add(book)
                  elif len([x for x in meta['features'] if x.startswith('talkingBook:')]) == 1:
                    lang = [x.split(':')[1] for x in meta['features'] if x.startswith('talkingBook:')][0]
                    fileid = hit.get_attribute_list('id')[0]
                    text = hit.get_text().strip()
                    
              
                if text != '':
                  url = f'https://s3.amazonaws.com/bloomharvest/{downsrc}/bloomdigital/audio/{fileid}.mp3'
                  if url not in urls:
                    page = requests.get(url)
                    if page.status_code == 200:
                      if not bloom_audio.get(lang):
                        bloom_audio[lang]=[] 
                      bloom_audio[lang].append({"book": book, "url": url, "text": text})
                      urls.append(url)
                    else:
                      badurls.append(url)
                      if not statcodes.get(page.status_code):
                        statcodes[page.status_code] = 0
                      statcodes[page.status_code] += 1
                      if not prob_books.get(book):
                        prob_books[book] = set()
                      prob_books[book].add(url)
                  else:
                    if {"book": book, "url": url, "text": text} not in bloom_audio.get(lang,[]):
                      if not bloom_audio.get(lang):
                        bloom_audio[lang]=[] 
                      bloom_audio[lang].append({"book": book, "url": url, "text": text})

            except:
              print(print(f'{title_path} failed download'))
              raise
            break

        with open('/content/drive/MyDrive/bloom/valid_urls.pkl', 'wb') as pklfile:
          pickle.dump(urls, pklfile)
                      
          #print(book)

      

        

In [ ]:
with open('/content/drive/MyDrive/bloom/all-lang-files-v2.json', 'w', encoding='utf-8') as outfile:
  json.dump(bloom_audio,outfile, ensure_ascii=False, indent=4)

In [ ]:
with open('/content/drive/MyDrive/bloom/all-lang-files-v2.json', 'r', encoding='utf-8') as infile:
  bloom_audio = json.load(infile)

In [ ]:
print(statcodes)

In [ ]:
if {"book": book, "url": url, "text": text} not in bloom_audio.get(lang,[]):
  print(book)

In [ ]:
resource_list = [f'{x}: {len(y)}' for x,y in bloom_audio.items()]
resource_list.sort()
print(resource_list)

In [ ]:
print(len(bloom_audio.keys()))

In [ ]:
bloom_audio['nsn'][11]

In [ ]:
page = requests.get(bloom_audio['nsn'][11]['url'])

In [ ]:
page.content

In [ ]:
tag_to_iso.get('en')

In [ ]:
bloom_dir = '/content/drive/MyDrive/bloom/bloom_downloads'
books = os.listdir(bloom_dir)
book_info = {}

for lang in tqdm(list(bloom_audio.keys())):
    tag = lang.split('-')[0]
    iso = tag_to_iso.get(tag)
    if not book_info.get(iso):
      book_info[iso]={}
    if iso:
      for book in {x['book'] for x in bloom_speech[iso]}:
        metapath = os.path.join(bloom_dir, book, 'meta.json')
        if os.path.isfile(metapath):
          with open(metapath, 'rb') as jsonfile:
            meta = json.load(jsonfile)
          license = meta.get('license')
          copyright = meta.get('copyright')
          credits = meta.get('credits')
          instance = meta.get('bookInstanceId')
          book_info[iso][book] = {'bloomLanguageTag': lang, 'license': license, 'copyright':copyright, 'credits':credits, 'instance':instance}

with open('/content/drive/MyDrive/bloom/book_info_v2.json', 'w', encoding='utf-8') as jsonfile:
  json.dump(book_info, jsonfile)

In [ ]:
bloom_speech['eng']

In [ ]:
with open('/content/drive/MyDrive/bloom/book_info.json', 'r', encoding='utf-8') as jsonfile:
  book_info = json.load(jsonfile)

In [ ]:
import json

In [ ]:
with open('/content/drive/MyDrive/bloom/charmapping.json') as f:
  charmap = json.load(f)

def get_script(x, charmap):
  scripts = []
  for c in list(x):
    if str(ord(c)) in charmap.keys():
      scripts.append(charmap[str(ord(c))])
  if len(scripts) > 0:
    return(set(scripts))
    #return max(set(scripts), key = scripts.count) 
  else:

    return "Unknown"

In [ ]:
get_script('''하나님께서 아기들을 죽이라는 바로의 명령을 따르지 않은 두 산파들을 기뻐하셔서 그들에게도 자녀의 축복을 주셨습니다.''', charmap)

In [ ]:
lang_scripts = {}
for lang in tqdm(list(bloom_audio.keys())):
  lang_scripts[lang] =  {}
  for itm in bloom_audio[lang]:
    scr = get_script(itm['text'], charmap)
    if not lang_scripts[lang].get(scr):
      lang_scripts[lang][scr] = 0
    lang_scripts[lang][scr] += 1

print(lang_scripts)

In [ ]:
for lang in tqdm(list(lang_scripts.keys())):
  mainscript = [x for x in lang_scripts[lang].keys() if lang_scripts[lang][x] == max(list(lang_scripts[lang].values()))][0]
  print(f'{lang} uses {mainscript}')
  for itm in bloom_audio[lang]:
    if mainscript == get_script(itm['text'], charmap):
      print(f'Here is an example: \n✅ {itm["text"]}')
      break
  if len(list(lang_scripts[lang].keys()))>1:
    print('Here are counterexamples:')
    for itm in bloom_audio[lang]:
      if mainscript != get_script(itm['text'], charmap):
        print('❌ ' + itm['text'])
        if 'If you need somewhere to put more information about the book, you can use this page, which is the outside of the back cover.' in itm['text']:
          print(itm['url'])

In [ ]:
!wget http://ldml.api.sil.org/langtags.json /content/langtags.json

In [ ]:
with open('langtags.json', 'r', encoding='utf-8') as jsonfile:
  langtags = json.load(jsonfile)

In [ ]:
langtags[10]

In [ ]:
with open('/content/drive/MyDrive/bloom/bloom_speech_aws.json', 'r', encoding='utf-8') as jsonfile:
  bloom_speech = json.load(jsonfile)

In [ ]:
print(set(list(bloom_speech.keys())) - set(biblnlp_langs))
print(len(set(list(bloom_speech.keys())) - set(biblnlp_langs)))

In [ ]:
!pip install fasttext
import fasttext

In [ ]:
lid = fasttext.load_model('/content/drive/MyDrive/bloom/lid.176.bin')

In [ ]:
pred = lid.predict(bloom_speech['eng'][0]['text'], k=5)

In [ ]:
pred[0][0]

In [ ]:
lang_predicts = {}
for lang in tqdm(list(bloom_speech.keys())):
  lang_predicts[lang] = {}
  for idx, rec in enumerate(bloom_speech[lang]):
    book = rec['book']
    pred = lid.predict(rec['text'].replace('\n', ' '), k=5)
    for i in range(0,len(pred[0])):
      tag = pred[0][i].split('_')[-1]
      label = tag_to_iso.get(tag,tag)
      if not lang_predicts[lang].get(book):
        lang_predicts[lang][book] = {}
      if not lang_predicts[lang][book].get(label):
        lang_predicts[lang][book][label] = []
      lang_predicts[lang][book][label].append(pred[1][i])


In [ ]:
lang_scripts = {}
for lang in tqdm(list(bloom_speech.keys())):
  lang_scripts[lang] =  {}
  for itm in bloom_speech[lang]:
    scr = get_script(itm['text'], charmap)
    if not lang_scripts[lang].get(scr):
      lang_scripts[lang][scr] = 0
    lang_scripts[lang][scr] += 1

print(lang_scripts)

In [ ]:
from matplotlib import pyplot as plt



In [ ]:
def removenewline(input):
  return(input.replace('\n', ' '))

In [ ]:
predslist =[]
lplist = []
qbooks = {}

langlist = list(lang_predicts.keys())
langlist.sort()

with open('language_cleaning.txt', 'w', encoding = 'utf-8') as outfile:
  for lang in langlist:
    lpred = {x:[] for y in lang_predicts[lang].keys() for x in lang_predicts[lang][y].keys()}
    for book in lang_predicts[lang].keys():
      for label in lang_predicts[lang][book].keys():
        lpred[label].extend(lang_predicts[lang][book][label])
    maxlpred = max([len(lpred[x]) for x in lpred.keys()])
    maxlpid= [x for x in lpred.keys() if len(lpred[x]) == maxlpred][0]
    print(f'{lang} most frequently predicts {maxlpid} with average score of {sum(lpred[maxlpid])/len(lpred[maxlpid])}')
    outfile.write(f'{lang} most frequently predicts {maxlpid} with average score of {sum(lpred[maxlpid])/len(lpred[maxlpid])}\n')
    predslist = []
    for book in lang_predicts[lang].keys():
      maxpred = max([len(lang_predicts[lang][book][x]) for x in lang_predicts[lang][book].keys()])
      maxpid= [x for x in lang_predicts[lang][book].keys() if len(lang_predicts[lang][book][x]) == maxpred][0]
      avgscore = sum(lang_predicts[lang][book][maxpid])/len(lang_predicts[lang][book][maxpid])
      if lang != maxpid and avgscore > 0.45:
        predslist.append(f'\t{lang} {book} predicts {maxpid} with avg score:{sum(lang_predicts[lang][book][maxpid])/len(lang_predicts[lang][book][maxpid])}')
        if (maxpid == 'eng' and lang != 'eng') or (maxpid == 'spa' and lang != 'spa') or (lang == 'spa' and maxpid != 'spa') or (lang=='eng' and maxpid !='eng'):
          if not qbooks.get(lang):
            qbooks[lang] = []
          qbooks[lang].append(book)
      else:
        if lang == 'eng' and maxpid != 'eng' and any([x > 0.6 for x in lang_predicts[lang][book][maxpid]]):
          for rec in bloom_speech[lang]:
            if rec['book'] == book:
              pred = lid.predict(rec['text'].replace('\n', ' '), k=5)
              tag = pred[0][0].split('_')[-1]
              label = tag_to_iso.get(tag,tag)
              if label != 'eng' and pred[1][0] > 0.6:
                print(f'\t{lang} {book} {label} outlier: {removenewline(rec["text"])} score: {pred[1][0]}')
                outfile.write(f'\t{lang} {book} {label} outlier: {removenewline(rec["text"])} score: {pred[1][0]}\n')
        elif lang == 'spa' and maxpid != 'spa' and any([x > 0.6 for x in lang_predicts[lang][book][maxpid]]):
          for rec in bloom_speech[lang]:
            if rec['book'] == book:
              pred = lid.predict(rec['text'].replace('\n', ' '), k=5)
              tag = pred[0][0].split('_')[-1]
              label = tag_to_iso.get(tag,tag)
              if label != 'spa' and pred[1][0] > 0.6:
                print(f'\t{lang} {book} {label} outlier: {removenewline(rec["text"])} score: {pred[1][0]}')
                outfile.write(f'\t{lang} {book} {label} outlier: {removenewline(rec["text"])} score: {pred[1][0]}\n')
        elif lang != 'spa' and lang != 'eng' and (maxpid == 'eng' or maxpid == 'spa') and any([x > 0.6 for x in lang_predicts[lang][book][maxpid]]):
          for rec in bloom_speech[lang]:
            if rec['book'] == book:
              pred = lid.predict(rec['text'].replace('\n', ' '), k=5)
              tag = pred[0][0].split('_')[-1]
              label = tag_to_iso.get(tag,tag)
              if (label == 'spa' or label == 'eng') and pred[1][0] > 0.6:
                print(f'\t{lang} {book} {label} outlier: {removenewline(rec["text"])} score: {pred[1][0]}')
                outfile.write(f'\t{lang} {book} {label} outlier: {removenewline(rec["text"])} score: {pred[1][0]}\n')




    if predslist != []:
      predslist.sort()
      for line in predslist:
        print(line)
        outfile.write(line + '\n')

In [ ]:
for lang in list(book_info.keys()):
  tag = lang.split('-')[0]
  iso = tag_to_iso.get(tag)
  if iso == None:
    book_info.pop(lang)
  elif iso != lang:
    if not book_info.get(iso):
      book_info[iso] = {}
    book_info[iso].update(book_info.pop(lang))



In [ ]:
for lang in qbooks:
  langbookset = {x['book'] for x in bloom_speech[lang]}
  print(f'{lang}:{len(qbooks[lang])} of {len(list(langbookset))}')


In [ ]:
for lang in book_info.keys():
  for book in book_info[lang].keys():
    if book in qbooks.get(lang,[]):
      book_info[lang][book]['quarantine'] = True
    else:
      book_info[lang][book]['quarantine'] = False


In [ ]:
book_info['tpi']

In [ ]:
for lang in list(set(book_info.keys())-set(bloom_speech.keys())):
  book_info.pop(lang)

In [ ]:
book_info.keys()

In [ ]:
bi_sort = list(book_info.keys())
bi_sort.sort()
print(bi_sort)

In [ ]:
print(set(book_info.keys())-set(bloom_speech.keys()))
print(set(bloom_speech.keys())-set(book_info.keys()))

In [ ]:
book_info['bam']

In [ ]:
%%capture
!pip install datasets
import datasets

In [ ]:
datasets.load_dataset('')

In [ ]:
bloom_speech['bis']

In [ ]:
type(book_info['ajz'])

In [ ]:
book_info['ajz']

In [ ]:
lpred[maxlpid]

In [ ]:
plt.hist(lang_predicts['eng']['eng'])

In [ ]:
for rec in bloom_speech['bjn']:
  pred = lid.predict(rec['text'].replace('\n', ' '), k=5)
  tag = pred[0][0].split('_')[-1]
  label = tag_to_iso.get(tag,tag)
  if label == 'eng' and float(pred[1][0]) > 0.5:
    print(rec['book'], rec['text'].replace('\n', ' '))
  

In [ ]:
langtag_dict = {x['full']:x['iso639_3'] for x in langtags if x.get('full') and x.get('iso639_3')}
shorttag_dict = {x['full']:x['tag'] for x in langtags if x.get('full') and x.get('tag')}
tag_to_iso = {x['tag']:x['iso639_3'] for x in langtags if x.get('iso639_3') and x.get('tag')}
iso_to_tag = {x['iso639_3']:x['tag'].split('-')[0] for x in langtags if x.get('iso639_3') and x.get('tag')}

In [ ]:
with open('/content/drive/MyDrive/bloom/bloom_speech_aws.json', 'r', encoding='utf-8') as jsonfile:
  bloom_speech = json.load(jsonfile)

In [ ]:
with open('/content/drive/MyDrive/bloom/book_info.json', 'r', encoding='utf-8') as jsonfile:
  book_info = json.load(jsonfile)

In [ ]:
biblnlp_langs = ['aau', 'aaz', 'abx', 'aby', 'acf', 'acu', 'adz', 'aey', 'agd', 'agg', 'agm', 'agn', 'agr', 'agu', 'aia', 'ake', 'alp', 'alq', 'als', 'aly', 'ame', 'amk', 'amp', 'amr', 'amu', 'anh', 'anv', 'aoi', 'aoj', 'apb', 'apn', 'apu', 'apy', 'arb', 'arl', 'arn', 'arp', 'aso', 'ata', 'atb', 'atd', 'atg', 'auc', 'aui', 'auy', 'avt', 'awb', 'awk', 'awx', 'azg', 'azz', 'bao', 'bbb', 'bbr', 'bch', 'bco', 'bdd', 'bea', 'bel', 'bgs', 'bgt', 'bhg', 'bhl', 'big', 'bjr', 'bjv', 'bkd', 'bki', 'bkq', 'bkx', 'bla', 'blw', 'blz', 'bmh', 'bmk', 'bmr', 'bnp', 'boa', 'boj', 'bon', 'box', 'bqc', 'bre', 'bsn', 'bsp', 'bss', 'buk', 'bus', 'bvr', 'bxh', 'byx', 'bzd', 'bzj', 'cab', 'caf', 'cao', 'cap', 'car', 'cav', 'cax', 'cbc', 'cbi', 'cbk', 'cbr', 'cbs', 'cbt', 'cbu', 'cbv', 'cco', 'ces', 'cgc', 'cha', 'chd', 'chf', 'chk', 'chq', 'chz', 'cjo', 'cjv', 'cle', 'clu', 'cme', 'cmn', 'cni', 'cnl', 'cnt', 'cof', 'con', 'cop', 'cot', 'cpa', 'cpb', 'cpc', 'cpu', 'crn', 'crx', 'cso', 'cta', 'ctp', 'ctu', 'cub', 'cuc', 'cui', 'cut', 'cux', 'cwe', 'daa', 'dad', 'dah', 'ded', 'deu', 'dgr', 'dgz', 'dif', 'dik', 'dji', 'djk', 'dob', 'dwr', 'dww', 'dwy', 'eko', 'emi', 'emp', 'eng', 'epo', 'eri', 'ese', 'etr', 'faa', 'fai', 'far', 'for', 'fra', 'fuf', 'gai', 'gam', 'gaw', 'gdn', 'gdr', 'geb', 'gfk', 'ghs', 'gia', 'glk', 'gmv', 'gng', 'gnn', 'gnw', 'gof', 'grc', 'gub', 'guh', 'gui', 'gul', 'gum', 'guo', 'gvc', 'gvf', 'gwi', 'gym', 'gyr', 'hat', 'haw', 'hbo', 'hch', 'heb', 'heg', 'hix', 'hla', 'hlt', 'hns', 'hop', 'hrv', 'hub', 'hui', 'hus', 'huu', 'huv', 'hvn', 'ign', 'ikk', 'ikw', 'imo', 'inb', 'ind', 'ino', 'iou', 'ipi', 'ita', 'jac', 'jao', 'jic', 'jiv', 'jpn', 'jvn', 'kaq', 'kbc', 'kbh', 'kbm', 'kdc', 'kde', 'kdl', 'kek', 'ken', 'kew', 'kgk', 'kgp', 'khs', 'kje', 'kjs', 'kkc', 'kky', 'klt', 'klv', 'kms', 'kmu', 'kne', 'knf', 'knj', 'kos', 'kpf', 'kpg', 'kpj', 'kpw', 'kqa', 'kqc', 'kqf', 'kql', 'kqw', 'ksj', 'ksr', 'ktm', 'kto', 'kud', 'kue', 'kup', 'kvn', 'kwd', 'kwf', 'kwi', 'kwj', 'kyf', 'kyg', 'kyq', 'kyz', 'kze', 'lac', 'lat', 'lbb', 'leu', 'lex', 'lgl', 'lid', 'lif', 'lww', 'maa', 'maj', 'maq', 'mau', 'mav', 'maz', 'mbb', 'mbc', 'mbh', 'mbl', 'mbt', 'mca', 'mcb', 'mcd', 'mcf', 'mcp', 'mdy', 'med', 'mee', 'mek', 'meq', 'met', 'meu', 'mgh', 'mgw', 'mhl', 'mib', 'mic', 'mie', 'mig', 'mih', 'mil', 'mio', 'mir', 'mit', 'miz', 'mjc', 'mkn', 'mks', 'mlh', 'mlp', 'mmx', 'mna', 'mop', 'mox', 'mph', 'mpj', 'mpm', 'mpp', 'mps', 'mpx', 'mqb', 'mqj', 'msb', 'msc', 'msk', 'msm', 'msy', 'mti', 'muy', 'mva', 'mvn', 'mwc', 'mxb', 'mxp', 'mxq', 'mxt', 'myu', 'myw', 'myy', 'mzz', 'nab', 'naf', 'nak', 'nay', 'nbq', 'nca', 'nch', 'ncj', 'ncl', 'ncu', 'ndj', 'nfa', 'ngp', 'ngu', 'nhg', 'nhi', 'nho', 'nhr', 'nhu', 'nhw', 'nhy', 'nif', 'nin', 'nko', 'nld', 'nlg', 'nna', 'nnq', 'not', 'nou', 'npl', 'nsn', 'nss', 'ntj', 'ntp', 'nwi', 'nyu', 'obo', 'ong', 'ons', 'ood', 'opm', 'ote', 'otm', 'otn', 'otq', 'ots', 'pab', 'pad', 'pah', 'pao', 'pes', 'pib', 'pio', 'pir', 'pjt', 'plu', 'pma', 'poe', 'poi', 'pon', 'poy', 'ppo', 'prf', 'pri', 'ptp', 'ptu', 'pwg', 'quc', 'quf', 'quh', 'qul', 'qup', 'qvc', 'qve', 'qvh', 'qvm', 'qvn', 'qvs', 'qvw', 'qvz', 'qwh', 'qxh', 'qxn', 'qxo', 'rai', 'rkb', 'rmc', 'roo', 'rop', 'rro', 'ruf', 'rug', 'rus', 'sab', 'san', 'sbe', 'seh', 'sey', 'sgz', 'shj', 'shp', 'sim', 'sja', 'sll', 'smk', 'snc', 'snn', 'sny', 'som', 'soq', 'spa', 'spl', 'spm', 'sps', 'spy', 'sri', 'srm', 'srn', 'srp', 'srq', 'ssd', 'ssg', 'ssx', 'stp', 'sua', 'sue', 'sus', 'suz', 'swe', 'swh', 'swp', 'sxb', 'tac', 'tav', 'tbc', 'tbl', 'tbo', 'tbz', 'tca', 'tee', 'ter', 'tew', 'tfr', 'tgp', 'tif', 'tim', 'tiy', 'tke', 'tku', 'tna', 'tnc', 'tnn', 'tnp', 'toc', 'tod', 'toj', 'ton', 'too', 'top', 'tos', 'tpt', 'trc', 'tsw', 'ttc', 'tue', 'tuo', 'txu', 'ubr', 'udu', 'ukr', 'uli', 'ura', 'urb', 'usa', 'usp', 'uvl', 'vid', 'vie', 'viv', 'vmy', 'waj', 'wal', 'wap', 'wat', 'wbp', 'wed', 'wer', 'wim', 'wmt', 'wmw', 'wnc', 'wnu', 'wos', 'wrk', 'wro', 'wsk', 'wuv', 'xav', 'xed', 'xla', 'xnn', 'xon', 'xsi', 'xtd', 'xtm', 'yaa', 'yad', 'yal', 'yap', 'yaq', 'yby', 'ycn', 'yka', 'yml', 'yre', 'yuj', 'yut', 'yuw', 'yva', 'zaa', 'zab', 'zac', 'zad', 'zai', 'zaj', 'zam', 'zao', 'zar', 'zas', 'zat', 'zav', 'zaw', 'zca', 'zia', 'ziw', 'zos', 'zpc', 'zpl', 'zpo', 'zpq', 'zpu', 'zpv', 'zpz', 'zsr', 'ztq', 'zty', 'zyp']
print(len(biblnlp_langs))

In [ ]:
lics = set()
lang_counts = {}
rem_books = set()
rem_langs = set()
for lang in book_info:
  lang_counts[lang] = {'all':0, 'nd':0}
  for key in book_info[lang].keys():
    lang_counts[lang]['all']+=1
    lics.add(book_info[lang][key]['license'])
    if book_info[lang][key]['license'] == 'cc-by-nc-nd':
      print(key)
      lang_counts[lang]['nd']+=1
      rem_books.add(key)

print(lics)
print(lang_counts)
for lang in lang_counts:
  if lang_counts[lang]['all'] == lang_counts[lang]['nd']:
    print(f'remove {lang}')
    rem_langs.add(lang)

In [ ]:
for lang in book_info:
  if lang in rem_langs

In [ ]:
langlist = []
dict2to3 = {}
for lang in bloom_speech:
  langlist.append(lang)
  tag = iso_to_tag.get(lang, lang)
  if len(tag) == 2:
    langlist.append(tag)
    dict2to3.update({tag:lang})

langlist.sort()
print(langlist)
print(dict2to3)

In [ ]:
print(book_info['ajz'])

In [ ]:
print(len(langlist))

In [ ]:
for lang in bloom_audio.keys():
  tag = lang.split('-')[0]
  print(f'{tag} - {tag_to_iso.get(tag)}')

In [ ]:
bloom_speech = {}
lang_texts = {}
for lang in tqdm(list(bloom_audio.keys())):
  tag = lang.split('-')[0]
  iso = tag_to_iso.get(tag)
  if not bloom_speech.get(iso) and iso is not None:
    bloom_speech[iso] = []
    lang_texts[iso] = set()
  mainscript = [x for x in lang_scripts[lang].keys() if lang_scripts[lang][x] == max(list(lang_scripts[lang].values()))][0]
  for itm in bloom_audio[lang]:
    txt = itm['text']
    if 'If you need somewhere to put more information about the book, you can use this page, which is the outside of the back cover.' in txt:
      txt = txt.split('If you need somewhere to put more information about the book, you can use this page, which is the outside of the back cover.')[0]
    if any([x.isdigit() for x in itm['text']]):
      hasdigit = True
    else:
      hasdigit = False
    if iso is not None and (mainscript == get_script(txt, charmap) or all([x.isdigit() for x in itm['text']])):
      if txt not in lang_texts[iso]:
        bloom_speech[iso].append({'book':itm['book'], 'text': txt, 'url':itm['url'], 'containsNumber': hasdigit, 'bloomLanguageTag': lang})
        lang_texts[iso].add(txt)

with open('/content/drive/MyDrive/bloom/bloom_speech.json', 'w', encoding='utf-8') as jsonfile:
  json.dump(bloom_speech,jsonfile)

In [ ]:
speech_counts = {}
for lang in bloom_speech:
  speech_counts[lang] = len(list(bloom_speech[lang]))

bcl = [f'{k}:{v}' for k,v in speech_counts.items()]
bcl.sort()
print(bcl)

In [ ]:
lis = [1,2,3]
print(lis[-0:])

In [ ]:
used_keys = set()
failed_urls = {}
for lang in tqdm(list(bloom_speech.keys())):
  for idx, rec in enumerate(bloom_speech[lang]):
    i = 0
    filename = f'{lang}-0000-{rec["url"].split("/")[-1]}'
    while filename in used_keys:
      i += 1
      filename = f'{lang}-{i:04d}-{rec["url"].split("/")[-1]}'
    page = requests.get(rec['url'])
    if page.status_code == 200:
      with open('temp.mp3', 'wb') as aud:
        aud.write(page.content)
      client.upload_file('temp.mp3','bloom-speech', f'audio/{filename}')
      bloom_speech[lang][idx]['url'] = f'https://s3.amazonaws.com/bloom-speech/audio/{filename}'
    else:
      failed_urls[rec['url']] = filename
      with open('/content/drive/MyDrive/bloom/failed_urls.json', 'w', encoding='utf-8') as jsonfile:
        json.dump(failed_urls, jsonfile)

with open('/content/drive/MyDrive/bloom/bloom_speech_aws.json', 'w', encoding='utf-8') as jsonfile:
  json.dump(bloom_speech, jsonfile)



In [ ]:
print(failed_urls)

In [ ]:
poplist = []
for key in failed_urls.keys():
  if key.startswith('https://s3'):
    poplist.append(key)

for key in poplist:
  failed_urls.pop(key)

print(len(list(failed_urls.keys())))

In [ ]:
for lang in tqdm(list(bloom_speech.keys())):
  for idx, rec in enumerate(bloom_speech[lang]):
    if failed_urls.get(rec['url']):
      page = requests.get(rec['url'])
      if page.status_code == 200:
        oldurl = rec['url']
        with open('temp.mp3', 'wb') as aud:
          aud.write(page.content)
        client.upload_file('temp.mp3','bloom-speech', f'audio/{filename}')
        bloom_speech[lang][idx]['url'] = f'https://s3.amazonaws.com/bloom-speech/audio/{filename}'
        failed_urls.pop(oldurl)
        with open('/content/drive/MyDrive/bloom/failed_urls.json', 'w', encoding='utf-8') as jsonfile:
          json.dump(failed_urls, jsonfile)
      else:
        print(f'{rec["url"]} failed again')

with open('/content/drive/MyDrive/bloom/bloom_speech_aws.json', 'w', encoding='utf-8') as jsonfile:
  json.dump(bloom_speech, jsonfile)
      


In [ ]:
client.upload_file('/content/drive/MyDrive/bloom/book_info.json', 'bloom-speech', 'book_info.json')
client.upload_file('/content/drive/MyDrive/bloom/bloom_speech_aws.json', 'bloom-speech', 'bloom_speech.json')

In [ ]:
book_info.keys()

In [ ]:
with open('/content/drive/MyDrive/bloom/book_info.json', 'w', encoding='utf-8') as jsonfile:
  json.dump(book_info, jsonfile)

In [ ]:
client.upload_file('/content/drive/MyDrive/bloom/book_info.json', 'bloom-speech', 'book_info.json')

In [ ]:
i = 1
print(f'{i:04d}')

In [ ]:
bloom_audio['qaa']

In [ ]:
book_counts = {}
for lang in book_info:
  book_counts[lang] = len(list(book_info[lang].keys()))

bcl = [f'{k}:{v}' for k,v in book_counts.items()]
bcl.sort()
print(bcl)

In [ ]:
for lang in tqdm(list(bloom_audio.keys())):
  urlset = {x['url'] for x in bloom_audio[lang]}
  fileset = {'/'.join(y['url'].split('/')[-4:]) for y in bloom_audio[lang]}
  if len(list(urlset)) != len(list(fileset)):
    reuse = 0   
    textdict = {x:set() for x in fileset}
    urldict = {x:set() for x in fileset}
    for rec in bloom_audio[lang]:
      textdict['/'.join(rec['url'].split('/')[-4:])].add(rec['text'])
      urldict['/'.join(rec['url'].split('/')[-4:])].add(rec['book'])
    for k,v in textdict.items():
      if len(list(v)) > 1:
        reuse += 1
        #print(k)
        print(textdict[k])
        print(urldict[k])
    if reuse == 0:
      print(f'{lang} ✅')
    else:
      print(f'{lang} ❌ urlset: {len(list(urlset))} fileset: {len(list(fileset))}. {reuse} reused.')  
    
  else:
    print(f'{lang} ✅')

In [ ]:
text2lang = {}
for lang in tqdm(list(bloom_audio.keys())):
  for rec in bloom_audio[lang]:
    if not text2lang.get(rec['text']):
      text2lang[rec['text']] = set()
    text2lang[rec['text']].add(lang)

for text in list(text2lang.keys()):
  if len(list(text2lang[text])) > 1:
    print(f'{text} appears in {list(text2lang[text])}')

In [ ]:
bloom_speech = {}
texts = {}
langurls = set()

for lang in tqdm(list(bloom_audio.keys()):
  texts[lang] = set()
  for rec in bloom_audio[lang]:
    if rec['url'] not in langurls:
      if rec['text'] not in texts[lang]:
        page = requests.get(rec['url'])
        if page.status_code == 200:
          with open('temp.mp3', 'wb') as aud:
            aud.write(page.content)
          client.upload_file



In [ ]:
instset = {}
instdict
for lang in book_info:
  instset[lang] = set()
  for key in book_info[lang].keys():
    if book_info[lang][key].get('instance') and book_info[lang][key].get('instance') in instset[lang]:
      print(f'{lang} {book_info[lang][key]["instance"]} repeats')
    else:
      instset[lang].add(book_info[lang][key].get('instance'))

In [ ]:
print(len(urls))

In [ ]:
print(len(set(urls)))

In [ ]:
print(len(set(badurls)))

In [ ]:
print(badurls[:10])

In [ ]:
with open('/content/drive/MyDrive/bloom/bad_urls.pkl', 'wb') as pklfile:
  pickle.dump(badurls, pklfile)

with open('/content/drive/MyDrive/bloom/bad_books.pkl', 'wb') as pklfile:
  pickle.dump(prob_books, pklfile)

In [ ]:
print(len(list(prob_books.keys())))

In [ ]:
good_books = [bloom_audio[x][y]['book'] for x in list(bloom_audio.keys()) for y in range(len(bloom_audio[x]))]

In [ ]:
print(set(prob_books.keys() - set(good_books)))

In [ ]:
book_set = set()
for lang in bloom_audio.keys():
  for record in bloom_audio[lang]:
    book_set.add(record.get('book'))

print(book_set)

In [ ]:
'Pagluto' in books

In [ ]:
page = requests.get('https://bloomlibrary.org/talking-books')

In [ ]:
with open('/content/talking_books.html', 'r') as htmlfile:
  soup = BeautifulSoup(htmlfile, 'html.parser' )

In [ ]:
page.

In [ ]:
full_book_set = set()
for link in soup.findAll(attrs={'class':'css-1ywcju2'}):
  full_book_set.add(link.get_text().strip())

In [ ]:
print(len(list(full_book_set)), len(list(book_set)), len(list(full_book_set - book_set)))

In [ ]:
fb_list = list(full_book_set)
bs_list = list(book_set)
fb_list.sort()
bs_list.sort()
print(fb_list)
print(bs_list)

In [ ]:
print(full_book_set - book_set)
print(book_set - full_book_set)

In [ ]:
missing_books = []
notparsed_books = []
for book in list(full_book_set-book_set):
  if book not in books:
    missing_books.append(book)
  else:
    notparsed_books.append(book)


In [ ]:
print(len(missing_books), len(notparsed_books))

In [ ]:
bloom_dir = '/content/drive/MyDrive/bloom/bloom_downloads'
books = os.listdir(bloom_dir)

In [ ]:
def audiosent(tag):
  return tag.has_attr('audio-sentence')

In [ ]:
import json

tb_set = set()
tb_lds = {}
missing_lang = 0
has_lang = 0
ml_books = set()
hl_books = set()
lang_set = set()

for book in tqdm(books):
  metapath = os.path.join(bloom_dir, book, 'meta.json')
  if os.path.isfile(metapath):
    with open(metapath, 'rb') as jsonfile:
      meta = json.load(jsonfile)
    if meta.get('features'):
      for feat in meta['features']:
        if feat.startswith('talkingBook'):
          tb_set.add(feat)
          if not tb_lds.get(feat):
            tb_lds[feat] = set()
          langs = list(meta.get('language-display-names',{}).keys())
          for lang in langs:
            tb_lds[feat].add(lang)

          titles = [x for x in os.listdir(os.path.join(bloom_dir, book)) if x.endswith('.htm')]
          if titles == []:
            print(f'{book} lacks .htm')
            print(os.listdir(os.path.join(bloom_dir, book)))
          else:
            title=titles[0]
            title_path = os.path.join(bloom_dir, book, title)
            with open(title_path,'rb') as htmlfile:
              soup = BeautifulSoup(htmlfile, 'html.parser')
            for hit in soup.find_all(attrs={'class' : 'audio-sentence'}):
              if hit != []:
                if hit.attrs.get('lang') and hit.attrs.get('lang')!= '*':
                  lang = hit.attrs.get('lang')
                  id = hit.get_attribute_list('id')[0]
                  text = hit.get_text().strip()
                  has_lang += 1
                  lang_set.add(lang)
                  hl_books.add(book)
                elif parents_havelang(hit): 
                  for parent in hit.parents:
                    if parent.attrs.get('lang') and parent.attrs.get('lang') != '*':
                      lang = parent.attrs.get('lang')
                      break
                  id = hit.get_attribute_list('id')[0]
                  text = hit.get_text().strip()
                  has_lang += 1
                  lang_set.add(lang)
                  hl_books.add(book)
                #elif len(langs) == 1:
                  #lang = langs[0]
                  #id = hit.get_attribute_list('id')[0]
                  #text = hit.get_text().strip()
                  #has_lang += 1
                  #lang_set.add(lang)
                  #hl_books.add(book)
                elif len([x for x in meta['features'] if x.startswith('talkingBook:')]) == 1:
                  lang = [x.split(':')[1] for x in meta['features'] if x.startswith('talkingBook:')][0]
                  id = hit.get_attribute_list('id')[0]
                  text = hit.get_text().strip()
                  has_lang += 1
                  lang_set.add(lang)
                  hl_books.add(book)
                else:
                  missing_lang += 1
                  ml_books.add(book)

          

print(len(list(tb_set)))
    

In [ ]:
for parent in hit.parents:
  if parent.attrs.get('lang'):
    print(parent.attrs.get('lang')
  else:
    print(parent.name, parent.attrs, parent.attrs.get('lang'))

In [ ]:
print(missing_lang)
print(has_lang)
print(len(list(lang_set)))
print(len(list(ml_books-hl_books)))

In [ ]:
print(lang_set)

In [ ]:
lang_valid_set = {x.split('-')[0] for x in list(lang_set)}

In [ ]:
lang_lower_set = {x.lower() for x in list(lang_set)}

In [ ]:
print(len(lang_valid_set), len(lang_lower_set))

In [ ]:
a = 'blah'
print(a.split('-')[0])

In [ ]:
print(text)

In [ ]:
tb_list = list(tb_set)
tb_list.sort()
print(tb_list)

In [ ]:
print(tb_lds['talkingBook'])

In [ ]:
tb_tagged = set([x.split(':')[1] for x in tb_lds.keys() if ':' in x])

In [ ]:
tb_out = list(tb_lds['talkingBook']-tb_tagged)
tb_out.sort()
print(tb_out)

In [ ]:
!ls '/content/drive/MyDrive/bloom/bloom_downloads/Ang Pamilya/'

In [ ]:
!cat '/content/drive/MyDrive/bloom/bloom_downloads/Ang Pamilya/meta.json'

#Missing Files checks

In [ ]:
licexl = set()
nondisplay = set()
noaudio = set()
noretrieve = set()
nolangtags = set()
notextintags = set(notparsed_books)
book_aud = {}
for book in tqdm(notparsed_books):
  book_aud[book] = {}
  metapath = os.path.join(bloom_dir, book, 'meta.json')
  if os.path.isfile(metapath):
    with open(metapath, 'rb') as jsonfile:
      meta = json.load(jsonfile)
      if meta.get('license') in ['custom', 'ask', None]:
        licexl.add(book)
      downsrc = meta.get('downloadSource')
  else:
    print(f'No metadata for {book}')
  titles = [x for x in os.listdir(os.path.join(bloom_dir, book)) if x.endswith('.htm')]
  if titles == []:
    print(f'{book} lacks .htm')
    print(os.listdir(os.path.join(bloom_dir, book)))
  else:
    title=titles[0]
    title_path = os.path.join(bloom_dir, book, title)
    with open(title_path,'rb') as htmlfile:
      soup = BeautifulSoup(htmlfile, 'html.parser')
    if len(soup.findAll(attrs={'class' : 'audio-sentence'})) == 0:
      noaudio.add(book)
    else:
      if not any([x.get_attribute_list('lang',False) for x in soup.findAll(attrs={'class': 'audio-sentence'})]):
        nolangtags.add(book)
  #foundfileids = [x.split('/')[-1].split('.')[0] for x in urls]
  #fileids = [y.get_attribute_list('id')[0] for y in soup.findAll(attrs={'class' : 'audio-sentence'})]
  #if len(set(fileids)-set(foundfileids)) == len(set(fileids)):
  #  noretrieve.add(book)
  if not meta.get('language-display-names'):
    nondisplay.add(book)
  else:
    for lang in meta['language-display-names'].keys():
      book_aud[book][lang] = False
      for hit in soup.findAll(attrs={'lang': lang, 'class' : 'audio-sentence'}):
        fileid = hit.get_attribute_list('id')[0]
        text = hit.get_text().strip()                 
        if text != '':
          try:
            notextintags.remove(book)
          except:
            pass
          url = f'https://s3.amazonaws.com/bloomharvest/{downsrc}/bloomdigital/audio/{fileid}.mp3'
        
          page = requests.get(url)
          if page.status_code == 200:
            book_aud[book][lang] = True
            break
      for lhit in soup.findAll(attrs={'lang':lang}):
        for hit in lhit.findAll(attrs={'class' : 'audio-sentence'}):
          fileid = hit.get_attribute_list('id')[0]
          text = hit.get_text().strip()                 
          if text != '':
            try:
              notextintags.remove(book)
            except:
              pass
            url = f'https://s3.amazonaws.com/bloomharvest/{downsrc}/bloomdigital/audio/{fileid}.mp3'
          
            page = requests.get(url)
            if page.status_code == 200:
              book_aud[book][lang] = True
              break
    if not any([book_aud[book][x] for x in list(book_aud[book].keys())]):
      noretrieve.add(book)  
                                                                 

print(f'{len(licexl)} excluded because of license')
print(f'{len(nondisplay)} lack language-display-names in metadata')
print(f'{len(noaudio)} don\'t have audio-sentence')
print(f'{len(noretrieve)} had no valid urls')
print(f'{len(nolangtags)} has no lang tags')
print(f'{len(noaudio.union(nondisplay, licexl, noretrieve))} excluded of {len(notparsed_books)} books')

In [ ]:
print(set(list(notparsed_books)) - noretrieve - noaudio - licexl - nondisplay)

In [ ]:
for tbook in set(list(notparsed_books)) - noretrieve:
  with open(os.path.join(bloom_dir, tbook, 'meta.json'), 'rb') as jsonfile:
    tmeta = json.load(jsonfile)
  if tmeta.get('language-display-names'):
    ld_keys = list(tmeta['language-display-names'].keys())
    audio_keys = [x for x in list(book_aud[tbook].keys()) if book_aud[tbook][x]]
    if any([x not in audio_keys for x in ld_keys]):
      print(f'{tbook} has audio keys {audio_keys} and ld keys {ld_keys}')


In [ ]:
blah = set([1,2,3])
blah.remove(1)
print(blah)

In [ ]:
print(set(fileids))
print(meta.get('downloadSource'))
testfileids = fileids[:5]
print(len(set(fileids)-set(testfileids)))
print(len(set(fileids)))


In [ ]:
metapath = '/content/bloom_downloads/Inyawa/meta.json'
title_path = '/content/bloom_downloads/Inyawa/Inyawa.htm'
lang = 'cak'
files=0

with open(metapath, 'rb') as jsonfile:
  meta = json.load(jsonfile)
  downsrc = meta.get('downloadSource')

with open(title_path,'rb') as htmlfile:
  soup = BeautifulSoup(htmlfile, 'html.parser')
#with open('/content/drive/MyDrive/bloom/all-lang-files.csv', 'a', encoding='utf-16') as csvfile:
  #writer = csv.writer(csvfile, dialect='excel', delimiter='\t')
for lhit in soup.findAll(attrs={'lang':'cak'}):
  for hit in lhit.findAll(attrs={'class' : 'audio-sentence'}):
    #print(hit)
    fileid = hit.get_attribute_list('id')[0]
    text = hit.get_text().strip()                
    if text != '':
      url = f'https://s3.amazonaws.com/bloomharvest/{downsrc}/bloomdigital/audio/{fileid}.mp3'
      page = requests.get(url)
      if page.status_code == 200:
        files += 1
        print(text)
        #if not bloom_audio.get(lang):
      #   bloom_audio[lang]=[] 
      # bloom_audio[lang].append({"book": book, "url": url, "text": text})
      #else:
      #  print(f'{url} failed for {text}')
print(files)

In [ ]:
from google.colab import files

files.download(os.path.join(bloom_dir,'Los tesoros del bosque/meta.json'))

In [ ]:
client.download_file('bloom-speech', 'bloom_speech.json', 'bloom_speech.json')

In [ ]:
client.download_file('bloom-speech', 'book_info.json', 'book_info.json')

In [ ]:
with open('bloom_speech.json', 'r') as f:
  df = json.load(f)

In [ ]:
with open('book_info.json', 'r') as f:
  meta = json.load(f)

In [ ]:
meta['eng']['Instead of the Rooster']

In [ ]:
df.keys()

In [ ]:
df['bam'][0]

In [ ]:
for lang in df.keys():
  lics = {}
  lictypes = set([meta[lang][x['book']]['license'] for x in df[lang]])
  for lic in lictypes:
    if not lics.get(lic):
      lics[lic] = 0
    lics[lic] += [meta[lang][x['book']]['license'] for x in df[lang]].count(lic)
  print(lang, lics)